In [4]:
from offerclassif.data import load_json
from random import sample
import plotly.express as px
import numpy as np
from collections import Counter

# Loading data

In [20]:
thesaurus = load_json('../data/external/job_thesaurus.json')
train = load_json('../data/external/offers_train.json')
test = load_json('../data/external/offers_test_sample.json')

20 loaded from ../data/external/job_thesaurus.json.
84 loaded from ../data/external/offers_train.json.
2 loaded from ../data/external/offers_test_sample.json.


# Exploration

## Thesaurus

In [6]:
sample(thesaurus, k=5)

[{'sector_internal_label': 'secteur ingenierie etudes et r et d',
  'id': '5b501c52e542527b6b74ca77c7dd1a6f',
  'internal_label': 'Ingénieur génie civil'},
 {'sector_internal_label': 'secteur informatique',
  'id': 'a0a93ff34e2519abeaa66906fa3477c8',
  'internal_label': 'Développeur mobile'},
 {'sector_internal_label': 'secteur btp',
  'id': '1363e391f03ac0af11df2b6ab1cbbfc4',
  'internal_label': 'Conducteur de travaux'},
 {'sector_internal_label': 'secteur ingenierie etudes et r et d',
  'id': 'a5fffadd29fdd1e48ade904ac4b13041',
  'internal_label': 'Ingénieur de recherche'},
 {'sector_internal_label': 'secteur ingenierie etudes et r et d',
  'id': '95a900a299fa1736d4b494ca5b6c627f',
  'internal_label': 'Ingénieur calcul mécanique'}]

In [21]:
Counter([i['sector_internal_label'] for i in thesaurus])

Counter({'secteur informatique': 7,
         'secteur commerce vente': 4,
         'secteur ingenierie etudes et r et d': 5,
         'secteur btp': 4})

In [22]:
{(i['sector_internal_label'], i['internal_label']) for i in thesaurus}

{('secteur btp', 'Charpentier'),
 ('secteur btp', 'Chef de chantier'),
 ('secteur btp', 'Conducteur de travaux'),
 ('secteur btp', 'Dessinateur projeteur'),
 ('secteur commerce vente', 'Chargé de clientele'),
 ('secteur commerce vente', 'Commercial'),
 ('secteur commerce vente', 'Vendeur'),
 ('secteur commerce vente', 'Vendeur télephonie mobile'),
 ('secteur informatique', 'Consultant Java'),
 ('secteur informatique', 'Consultant informatique'),
 ('secteur informatique', 'Développeur .NET'),
 ('secteur informatique', 'Développeur C++'),
 ('secteur informatique', 'Développeur Java'),
 ('secteur informatique', 'Développeur Python'),
 ('secteur informatique', 'Développeur mobile'),
 ('secteur ingenierie etudes et r et d', 'Data Scientist'),
 ('secteur ingenierie etudes et r et d', 'Ingénieur calcul'),
 ('secteur ingenierie etudes et r et d', 'Ingénieur calcul mécanique'),
 ('secteur ingenierie etudes et r et d', 'Ingénieur de recherche'),
 ('secteur ingenierie etudes et r et d', 'Ingénieu

## Train

In [23]:
sample(train, k=2)

[{'employments': [{'employment_id': 'b2788e1ab6be65f1c38bf7d23cd081f1'}],
  '_id': 'AXDb_t3lZyTF-SpR_dmJ',
  'description': "Kalixens RH recherche un commercial (H/F) en CDI pour son client, une entreprise à taille humaine, spécialisée dans la conception de machines spéciales.Vous aurez pour mission principale le développement du portefeuille clients de l'entreprise.A ce titre, vos missions seront les suivantes :- La gestion du portefeuille clients déjà existant en fonction de leurs attentes- La prospection et la veille de nouveaux marchés- Le suivi de la relation avec les clients- L'établissement de devis personnalisés, en fonction des besoins des prospects- L'établissement et la négociation de propositions commerciales- Le reporting de votre activité auprès de votre responsable hiérarchiqueVous aurez ensuite à développer de nouveaux marchés dans la zone Europe (Belgique et Allemagne).Le poste est à pourvoir en CDI.La rémunération est entre 30K€ et 35K€ bruts annuel + part variable en

### employment_id

In [25]:
{len(i['employments']) for i in train}

{1}

All the offers have only one `employment_id`

In [26]:
train_employment_id = [j['employment_id'] for i in train for j in i['employments']]
thesaurus_employment_id = {i['id'] for i in thesaurus}

In [27]:
sum([i not in thesaurus_employment_id for i in train_employment_id])

0

All `train_employment_id` are in `thesaurus`

In [28]:
sum([i not in train_employment_id for i in thesaurus_employment_id])

8

Some `thesaurus_employment_id` do not appears in train samples 

In [29]:
Counter(train_employment_id)

Counter({'a17138bebe5261cfbefcf29ba44a532d': 1,
         'a5fffadd29fdd1e48ade904ac4b13041': 9,
         '8a7a3f1bce0958ae9f090683e5a925b7': 11,
         '95a900a299fa1736d4b494ca5b6c627f': 5,
         'b1802cee3f64d16df4fba4f28a4b5fdb': 18,
         'c370440263c4e22d85e7f11602a99ac0': 4,
         'f14e469063a4e1c4f21db06628869453': 6,
         'eff8ffd576d5afb7bb0849b772c6dc8f': 9,
         '1cfcdd2696584bb53d7d64a76038a817': 8,
         'b2788e1ab6be65f1c38bf7d23cd081f1': 6,
         'e7fc3727e6709ce3651cce4c149156f9': 6,
         '8cec8662e6fcc3385614a461d738f556': 1})

### Description

In [30]:
desc_lengths = [len(i['description'].split(' ')) for i in train]

fig = px.histogram(x=desc_lengths, marginal='rug')
fig.update_layout(title_text='Distribution du nombre de mots dans la description')

In [31]:
train[np.argmax(desc_lengths)]

{'employments': [{'employment_id': 'b2788e1ab6be65f1c38bf7d23cd081f1'}],
 '_id': 'AXDb_odQWjtLgivJ_ISE',
 'description': "Description du poste : QUI TU ESTu es une personne passionnée par les gens, par le business et tu mets tout en œuvre pour améliorer les résultats et l'activité. Tu es déterminé à créer un IKEA adapté à l'avenir de la distribution multicanale et bien ancré dans le process de changement qui en découle, à créer de la valeur ajoutée pour le client et à le placer au centre des priorités. À ce poste, il te faut au moins 5 ans d'expérience en Marketing Digital ou équivalent. Tu as l'expérience du pilotage des résultats dans le périmètre assigné, ainsi que de leur suivi, et l'expérience de la distribution, de préférence dans le secteur numérique. Tu maîtrises les techniques de résolution de problèmes et de gestion des conflits comme les outils et méthodes de conduite du changement et de gestion des parties prenantes. Tu as de solides compétences d'organisation et de planifi

In [32]:
train[np.argmin(desc_lengths)]

{'employments': [{'employment_id': 'b1802cee3f64d16df4fba4f28a4b5fdb'}],
 '_id': 'AXDcBfBMZyTF-SpR_3-u',
 'description': 'Vous accueillez et conseillez les clients - assurez le passage de clients en caisse - proposez les services du magasin + ventes additionnelles - assurez la satisfaction des clients.',
 'title': 'Vendeur / Vendeuse en prêt-à-porter'}

### Title

In [33]:
title_lengths = [len(i['title'].split(' ')) for i in train]

fig = px.histogram(x=title_lengths)
fig.update_layout(title_text='Distribution du nombre de mots dans le titre')

## Test

In [34]:
test

[{'_id': 'AXDcDUvTuYbxoIaodd7C',
  'description': "Vous êtes en charge de l'accueil des clients, de la tenue du magasin et du conseil client. Vous êtes, également, en charge des encaissements. Vous avez une expérience confirmée sur un poste similaire. Poste pouvant évoluer sur la responsabilité du magasin.",
  'title': 'Vendeur / Vendeuse en prêt-à-porter'},
 {'_id': 'AXDb_jbaWjtLgivJ_Hbn',
  'description': "Gold Partner Microsoft (prix du partenaire de l'année Microsoft 2018) ayant pour moteur l'innovation autout de sujets tels que : Big Data, Machine Learning, Internet des Objets, Web et Services Mobiles, Bot Intelligents, Plateformes Cloud intelligentes. recherche dans le cadre du développement de leur activité un Développeur .NET Azure (Tous niveaux d'expérience acceptés - Salaire selon Profil)En tant que Développeur Microsoft .NET Azure, vous interviendrez dans le cadre de la mise en place de différents types de solutions sur la base des technologies Microsoft (les missions peuven